## Задание 1

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [152]:
import pandas as pd
import numpy as np

ratings = pd.read_csv('movies/ratings.csv')

(ratings
 .groupby('userId')
 .agg(
     user_live=('timestamp', np.ptp),
     rating_count=('rating', 'count'),
 )
 .query('rating_count > 100')
 ['user_live']
 .mean()
)

40080507.4496124

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [153]:
import pandas as pd

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [158]:
from typing import List
from functools import reduce

def merge_dfs(dfs: List[pd.DataFrame], **kwargs) -> pd.DataFrame:
    return reduce(lambda res, df: res.merge(df, **kwargs), dfs)

merged_data = merge_dfs([client_base, rzd, auto, air], on='client_id', how='outer')

In [159]:
# without address
merged_data.drop(columns='address')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [160]:
# with address
merged_data

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

### Решение

Если предположить, что данные о визите коррелируются с данными о местоположении и местоположение могло меняться в течение визита, то логично данные о местоположении представить в виде списка координат. Каждая координата - это tuple широты/долготы. Каждая пара координат в списке отображает процесс изменения местоположения пользователя.

Так же я бы:
- попытался продать бизнесу идею - раздавать новые айдишники каждому визиту пользователей, чтобы различать сессии;
- попытался продать бизнесу идею - определять время для визитов и обновления координат;
- объединенил таблиц по индексам user_id и visit_id;
- замапил совершение покупки на определенные координаты и время.

#### Какой от этого будет профит?

Я считаю, что для многих видов бизнеса будет полезным иметь карту действий своих пользователей в зависимости от местоположения и времени. Таким образом, бизнесу будет значительно проще повысить рентабельность инвестиций в рекламу, каналы продаж, они смогут лучше познакомиться со своей аудиторией и получат новые метрики, которые помогут им улучшить свой продукт.